In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/MyStudy/dlwpt-code-ja/p2ch10/

/content/drive/MyDrive/MyStudy/dlwpt-code-ja/p2ch10


In [ ]:
!pwd

/content/drive/MyDrive/MyStudy/dlwpt-code-ja/p2ch10


In [ ]:
%matplotlib inline
import numpy as np
import functools
import glob
import os
import csv

from .u

ModuleNotFoundError: ignored

# データの確認

In [ ]:
# ファイル内の行数を数える
!wc -l ../data-unversioned/part2/luna/candidates.csv

551066 ../data-unversioned/part2/luna/candidates.csv


In [ ]:
# ファイルの最初の数行を出力
# 一行目は列ヘッダー

!head ../data-unversioned/part2/luna/candidates.csv

seriesuid,coordX,coordY,coordZ,class
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,-56.08,-67.85,-311.92,0
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,53.21,-244.41,-245.17,0
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,103.66,-121.8,-286.62,0
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,-33.66,-72.75,-308.41,0
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,-32.25,-85.36,-362.51,0
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,-26.65,-203.07,-165.07,0
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,-74.99,-114.79,-311.92,0
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,-16.14,-248.61,-239.55,0
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,135.89,-141.41,-252.2,0


In [ ]:
! grep -E ',1$' ../data-unversioned/part2/luna/candidates.csv | wc -l

1351


In [ ]:
!wc -l ../data-unversioned/part2/luna/annotations.csv

1187 ../data-unversioned/part2/luna/annotations.csv


In [ ]:
! head ../data-unversioned/part2/luna/annotations.csv

seriesuid,coordX,coordY,coordZ,diameter_mm
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,-128.6994211,-175.3192718,-298.3875064,5.651470635
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,103.7836509,-211.9251487,-227.12125,4.224708481
1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793540579077826395208,69.63901724,-140.9445859,876.3744957,5.786347814
1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016233746780170740405,-24.0138242,192.1024053,-391.0812764,8.143261683
1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016233746780170740405,2.441546798,172.4648812,-405.4937318,18.54514997
1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016233746780170740405,90.93171321,149.0272657,-426.5447146,18.20857028
1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016233746780170740405,89.54076865,196.4051593,-515.0733216,16.38127631
1.3.6.1.4.1.14519.5.2.1.6279.6001.100953483028192176989979435275,81.50964574,54.9572186,-150.3464233,10.36232088
1.3.6.1.4.1.14519.5.2.1.6279.6001.10268196240

## 高階関数とは
* 関数を引数に取る関数、および、返り値が関数である関数

In [ ]:
# 関数を受け取り実行する関数を定義する
def logger(func):
    def inner(*args):
        print('引数:', *args)
        return func(*args)
    return inner

def accumulate(a, b):
    return(a, b)

newfunc = logger(accumulate)
print(newfunc(1, 2))

引数: 1 2
(1, 2)


## デコレータとは
* デコレータは関数やクラスの前後に特定の処理を追加できる機能
* 関数やクラス宣言の前に@デコレータ名を記述することで実現

以下のコード
```
@foo
def bar():
    pass

bar()

```
は実質的に以下と同義


```
foo(bar)()
```





In [ ]:
# 一つ前のコードをデコレータを使って表現すると
@logger # <- デコレータ
def accumulate(a, b):
    return a + b

print(accumulate(1, 2))

引数: 1 2
3


# 前処理

In [ ]:
from collections import namedtuple
CandidateInfoTuple = namedtuple(
    'CandidateInfoTuple',
    'isNodule_bool, diameter_mm, series_uid, center_xyz'
)

## 以下と同じ
# CandidateInfoTuple = namedtuple(
#     'CnadidateInfoTuple', ['isNodule_bool', 'diameter_mm', 'series_uid', 'center_xyz']
# )

In [ ]:
@functools.lru_cache(1)
def getCandidateInfoList(requireOnDisk_bool=True):
    """
    candidate.csvに結節の半径の情報を付与してリストとして返す関数
    """
    mhd_list = glob.glob('../data/part2/luna/subset*/*.mhd')
    presentOnDisk_set = {os.path.split(p)[-1][:-4] for p in mhd_list}

    diameter_dict = {}
    with open('../data-unversioned/part2/luna/annotations.csv', 'r') as f:
        for row in list(csv.reader(f))[1:]: # headerは飛ばす
            series_uid = row[0]
            annotationCenter_xyz = tuple([float(x) for x in row[1:4]])
            annotationDiameter_mm = float(row[4])

            diameter_dict.setdefault(series_uid, []).append((annotationCenter_xyz, annotationDiameter_mm))

    candidateInfo_list = []
    with open('../data-unversioned/part2/luna/candidates.csv', 'r') as f:
        for row in list(csv.reader(f))[1:]:
            series_uid = row[0]

            if series_uid not in presentOnDisk_set and requireOnDisk_bool:
                # candidates.csvの識別子と一致するmhdファイルがない & requireOnDis_bool==Trueだったら
                continue

            isNodule_bool = bool(int(row[4])) # 結節か
            candidateCenter_xyz = tuple([float(x) for x in row[1:4]])

            candidateDiameter_mm = 0.0
            for annotation_tup in diameter_dict.get(series_uid, []):
                annotationCenter_xyz, annotationDiameter_mm = annotation_tup
                for i in range(3):
                    delta_mm = abs(candidateCenter_xyz[i]-annotationCenter_xyz[i])
                    if delta_mm > annotationDiameter_mm / 4:
                        break # アノテーションの正常性をチェック, candidateのほうがannotationよりデータが多い
                else:
                    candidateDiameter_mm = annotationDiameter_mm
                    break

            candidateInfo_list.append(CandidateInfoTuple(
                isNodule_bool, candidateDiameter_mm, series_uid, candidateCenter_xyz
            ))

    candidateInfo_list.sort(reverse=True)
    return candidateInfo_list

In [ ]:
!pip install simpleitk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 14.0 MB/s eta 0:00:00


In [ ]:
import SimpleITK as sitk

class Ct:
    def __init__(self, series_uid):
        mhd_path = glob.glob('../data-unversioned/part2/luna/subset*/{}.mhd'.format(series_uid))[0]

        # .rawファイルも暗黙的に使用
        ct_mhd = sitk.ReadImage(mhd_path)

        # numpy配列に変換
        ct_a = np.array(sitk.GetArrayFromImage(ct_mhd), dtype=np.float32)
        ct_a.clip(-1000, 1000, ct_a)

        self.series_uid = series_uid
        self.hu_a = ct_a

        self.origin_xyz = XyzTuple(*ct_mhd.GetOrigin())
        self.vsSize_xyz = XyzTuple(*ct_mhd.GetSpacing())
        self.direction_a = np.array(ct_mhd.GetDirection()).reshape(3, 3)

SyntaxError: ignored

In [ ]:
a = [[[1, 2], [2, 3]], [[2, 3], [5, 13]], [[1, 3], [2, 8]]]
a

[[[1, 2], [2, 3]], [[2, 3], [5, 13]], [[1, 3], [2, 8]]]

In [ ]:
a[::-1]

[[[1, 3], [2, 8]], [[2, 3], [5, 13]], [[1, 2], [2, 3]]]

In [ ]:
import torch

torch.tensor([
    not True, True
],dtype=torch.long)

tensor([0, 1])

In [22]:
import logging

In [13]:
a = [(1, 40, 1, (234, 1241, 12)), (1, 55, 1, (234, 1241, 12)), (1, 20, 1, (234, 1241, 12)), (0, 55, 1, (234, 1241, 12)), (0, 55, 8, (234, 1241, 12))]

In [14]:
a.sort(reverse=True)

In [15]:
a

[(1, 55, 1, (234, 1241, 12)),
 (1, 40, 1, (234, 1241, 12)),
 (1, 20, 1, (234, 1241, 12)),
 (0, 55, 8, (234, 1241, 12)),
 (0, 55, 1, (234, 1241, 12))]

In [21]:
import glob
path = glob.glob('/content/drive/MyDrive/MyStudy/MyAtCoder/DP/*.py')
path[0]

'/content/drive/MyDrive/MyStudy/MyAtCoder/DP/Fibonacci.py'

In [23]:
import numpy as np
a = np.array([[1, 2, 4], [3, 2 ,4], [3, 1, 1]])
a_inv = np.linalg.inv(a)

In [24]:
a @ a_inv

array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [1.11022302e-16, 0.00000000e+00, 1.00000000e+00]])

In [25]:
a_inv

array([[-0.5 ,  0.5 ,  0.  ],
       [ 2.25, -2.75,  2.  ],
       [-0.75,  1.25, -1.  ]])

In [44]:
a = np.array([2, 1, 4])

In [45]:
IrcTuple = namedtuple('IrcTuple', ['index', 'row', 'col'])

In [46]:
a_tup = IrcTuple(a[2], a[1], a[0])

In [47]:
int(a[2])

4

In [48]:
import sys
sys.argv

['/usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py',
 '-f',
 '/root/.local/share/jupyter/runtime/kernel-d77dde7d-0e52-4511-8ea7-e1bfa92e9292.json']

In [59]:
import argparse

parser = argparse.ArgumentParser(description='この説明はなくていいよ')

parser.add_argument('arg1')
# parser.add_argument('--arg-aaa')

args = parser.parse_args() # 解析

print(args.arg1)

usage: ipykernel_launcher.py [-h] arg1
ipykernel_launcher.py: error: unrecognized arguments: -f


SystemExit: ignored